In [ ]:
import pandas as pd
import seaborn as sns
from __future__ import (absolute_import, division, print_function, unicode_literals)

import warnings
warnings.simplefilter('ignore')

%pylab inline
%config InlineBackend.figure_format = 'svg' 

from pylab import rcParams
rcParams['figure.figsize'] = 6,3

In [ ]:
data_set = pd.read_csv('nba_2013.csv')
data_set.shape

In [ ]:
data_set.dropna()
data_set.shape

In [ ]:
data_set.info()

In [ ]:
data_set.head()

###### (1) Строим гистограмму для визуализации распределения возраста игроков 
Используются переменная Age (Возраст) непрерывного типа.

Использована гистограмма, т.к. только с её помощью можно отобразить графически частоту распределения непрерывной
величины.

In [ ]:
age_distr = data_set.age.hist(bins=20)
age_distr.set_xlabel('Age')
age_distr.set_ylabel('Number of players')
age_distr.set_title('Players age distribution')

###### (2) Строим блок диаграмм рассеивания и диаграмм распределения для визуальной оценки парных корреляций и распределений
Использовались следующие переменые:

mp - количество минут проведенных игроком за сезон на игровой площадке - переменная непрерывного типа.

pts - количество набранных игроком за сезон очков - непрерывная переменная.

gs - количество игр, в которых игрок выходил на поле с первой минуты - дискретная перемнная.

Использовал этот тип графика т. к. позволяет отразить несколько парных корреляций сразу.


In [ ]:
sns.pairplot(data_set[['mp', 'pts', 'gs']])

###### (3) Строим Buble chart. который будет отображать зависимость среднего времени нахождения игроков на площадке от  среднего возраста с доролнительным измерением по сумме набранных очков.

Берем непрерывную величину времени нахождения на площадке, непрерывную величину возраст, категориальную величину Позиция и дискретную величину Количество набранных очков.

Buble chart в данном случае позволяет отразить сразу три показателя, а также зависимость между двумя из них.

In [ ]:
from plotly.offline import init_notebook_mode, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [ ]:
age_mp_position = data_set.groupby('pos')[['age', 'mp']].mean()
scores_position = data_set.groupby('pos')[['pts']].sum()

position_data = age_mp_position.join(scores_position)

position_data.head(7)
    


In [ ]:
trace0 = go.Scatter(x=position_data.age, y=position_data.mp, mode = 'markers+text', text = position_data.index,
         textposition='bottom', marker = dict(size = 1/1000 * position_data.pts, color = ['aqua', 'azure', 'beige',
         'lightgreen', 'lavender', 'plum', 'wheat']))

data = [trace0]
layout = {'title': 'Players statistics by position', 'xaxis': {'title': 'Age'}, 'yaxis': {'title': 'Minutes played'}}

fig = go.Figure(data=data, layout=layout)

iplot(fig, show_link=False)

###### (4) Построим Pie Chart для визуализации доли позиции игрока каждого вида в общей массе игроков лиги
Использовадись категориальная переменная pos (Наименование позиции) и категориальная переменная player (Имя/Фамилия игрока).

Думаю, что для визуализации долевой струтуры при небольшом наборе категорий Pie Chart подходит наилучшим образом.  

In [ ]:
team_members = data_set.groupby(['pos'])[['player']].count()

In [ ]:
team_members.head(10)

In [ ]:
labels = team_members.index.tolist()
values = team_members.player.tolist()

In [ ]:
trace = go.Pie(labels=labels, values=values)

iplot([trace], filename='basic_pie_chart')

###### (5) Построим горизонтальный Bar Chart для визуализации суммы нбранных каждой команды на фоне других.

Берем категориальную переменную Наименование команды и дискретную величину Количество набранных очков.

В данном случае Bar Chart позволяет наглядно отразить визуальное сравнение сравнительно большоко числа категорий по определенному параметру.

In [ ]:
scores_team = data_set.groupby('bref_team_id')[['pts']].sum()

In [ ]:
trace_1 = go.Bar(x=scores_team.pts, y=scores_team.index, orientation = 'h')

data = [trace_1]
layout = {'title': 'Scores by team name', 'xaxis': {'title': 'Pts'}, 'yaxis': {'title': 'Team name'}}

fig = go.Figure(data=data, layout=layout)

iplot(fig, show_link=False)